# 03 — IDPs (latest snapshot by province)

**Dataset (source file):** `raw_data/hdx_hapi_idps_afg.csv`  
**Geo boundary used by the website:** `raw_data/geo/afghanistan_adm1.geojson`  
**Website section:** `js/sections/idps.js`

This notebook:
1) reads raw IDPs data,
2) filters to `admin_level=1` and a valid date + positive population,
3) finds the latest snapshot date,
4) aggregates by province,
5) saves:
   - cleaned snapshot rows to `dataset/cleaned/idps_admin1_latest_clean.csv`
   - dropped rows to `dataset/cleaned/idps_dropped.csv`
   - aggregates to `dataset/derived/idps_latest_by_province.csv` + `dataset/derived/idps_latest_total.json`


In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

RAW_DIR = Path("raw_data")        # raw inputs live here
DATASET_DIR = Path("dataset")     # website reads from here
CLEAN_DIR = DATASET_DIR / "cleaned"
DERIVED_DIR = DATASET_DIR / "derived"

CLEAN_DIR.mkdir(parents=True, exist_ok=True)
DERIVED_DIR.mkdir(parents=True, exist_ok=True)

def to_number(s: pd.Series) -> pd.Series:
    return pd.to_numeric(s.astype(str).str.replace(",", "").str.strip(), errors="coerce")

def parse_year(val) -> float:
    if pd.isna(val):
        return np.nan
    s = str(val).strip()
    if len(s) >= 4 and s[:4].isdigit():
        return float(s[:4])
    for sep in ("/", "-"):
        parts = s.split(sep)
        if len(parts) == 3 and parts[-1].isdigit():
            return float(parts[-1])
    return np.nan

def show_filter_report(df_before: pd.DataFrame, mask: pd.Series, title: str, sample_n: int = 10):
    kept = df_before[mask].copy()
    dropped = df_before[~mask].copy()
    print(f"=== {title} ===")
    print("rows_before:", len(df_before))
    print("rows_kept:", len(kept))
    print("rows_dropped:", len(dropped))
    if len(dropped):
        print("\nSample dropped rows:")
        display(dropped.head(sample_n))
    return kept, dropped


In [6]:

df = pd.read_csv("hdx_hapi_idps_afg.csv")
df.head()



,location_code,has_hrp,in_gho,provider_admin1_name,provider_admin2_name,admin1_code,admin1_name,admin2_code,admin2_name,admin_level,operation,assessment_type,population,reporting_round,reference_period_start,reference_period_end
0,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,246362,1,2017-03-30,2017-03-30
1,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,945182,2,2017-06-30,2017-06-30
2,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1728157,3,2017-12-31,2017-12-31
3,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1843833,4,2018-03-30,2018-03-30
4,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1866925,5,2018-06-30,2018-06-30


In [8]:
required = {"admin_level","reference_period_start","population"}
missing = sorted(required - set(df.columns))
assert not missing, f"Missing columns: {missing}"

df2 = df.copy()
df2["admin_level_str"] = df2["admin_level"].astype(str).str.strip()
df2["date"] = pd.to_datetime(df2["reference_period_start"], errors="coerce")
df2["population_num"] = to_number(df2["population"])

mask = (df2["admin_level_str"] == "1") & df2["date"].notna() & df2["population_num"].notna() & (df2["population_num"] > 0)
kept, dropped = show_filter_report(df2, mask, "IDPs filter (admin_level=1, valid date, numeric pop, pop>0)")


=== IDPs filter (admin_level=1, valid date, numeric pop, pop>0) ===
rows_before: 5298
rows_kept: 425
rows_dropped: 4873

Sample dropped rows:


,location_code,has_hrp,in_gho,provider_admin1_name,provider_admin2_name,admin1_code,admin1_name,admin2_code,admin2_name,admin_level,operation,assessment_type,population,reporting_round,reference_period_start,reference_period_end,admin_level_str,date,population_num
0,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,246362,1,2017-03-30,2017-03-30,0,2017-03-30,246362
1,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,945182,2,2017-06-30,2017-06-30,0,2017-06-30,945182
2,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1728157,3,2017-12-31,2017-12-31,0,2017-12-31,1728157
3,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1843833,4,2018-03-30,2018-03-30,0,2018-03-30,1843833
4,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,1866925,5,2018-06-30,2018-06-30,0,2018-06-30,1866925
5,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,2465495,6,2018-09-30,2018-09-30,0,2018-09-30,2465495
6,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,3529971,7,2018-12-31,2018-12-31,0,2018-12-31,3529971
7,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,4191521,8,2019-06-30,2019-06-30,0,2019-06-30,4191521
8,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,4350900,9,2019-12-31,2019-12-31,0,2019-12-31,4350900
9,AFG,True,True,NaN,NaN,NaN,NaN,NaN,NaN,0,Returns and displacement in Afghanistan,BA,4760478,10,2020-06-30,2020-06-30,0,2020-06-30,4760478


In [10]:
latest_date = kept["date"].max()
assert pd.notna(latest_date), "No valid dates after filtering"
latest_iso = latest_date.strftime("%Y-%m-%d")

snap = kept[kept["date"] == latest_date].copy()
snap["admin1_name"] = snap.get("admin1_name", "").astype(str).str.strip()

snap_out = CLEAN_DIR / "idps_admin1_latest_clean.csv"
snap.to_csv(snap_out, index=False)
snap_out, latest_iso


(WindowsPath('dataset/cleaned/idps_admin1_latest_clean.csv'), '2022-12-31')

In [12]:
by_prov = (snap.groupby("admin1_name", as_index=False)["population_num"]
             .sum()
             .sort_values("population_num", ascending=False))
by_prov.rename(columns={"population_num":"idps_count"}, inplace=True)

out_csv = DERIVED_DIR / "idps_latest_by_province.csv"
by_prov.to_csv(out_csv, index=False)

total = float(by_prov["idps_count"].sum())
out_json = DERIVED_DIR / "idps_latest_total.json"
out_json.write_text(json.dumps({"latest_date": latest_iso, "idps_total_admin1": total}, indent=2), encoding="utf-8")

(out_csv, out_json)


(WindowsPath('dataset/derived/idps_latest_by_province.csv'),
 WindowsPath('dataset/derived/idps_latest_total.json'))

In [14]:
dropped_out = CLEAN_DIR / "idps_dropped.csv"
dropped.to_csv(dropped_out, index=False)
dropped_out


WindowsPath('dataset/cleaned/idps_dropped.csv')